# Double Entry Posting with Tax Accounts

The simplest case of double entry accounting is based on posting a transaction at least to two accounts. Document transactions with tax involve one additional tax account.

Accounting software typically provides a frontend dialog for typing in debit and credit account and amount and also a control code, which steer tax rate, tax acount and other bookkeeping functions. The following code simulates this process and posts transactions on two accounts plus an tax account. It also posts opening entries without tax. The result is a list of postings, which can be analyzed by account or grouped to a journal or trial balance.

In [39]:
import pandas as pd
import numpy as np

# Use 3 decimal places in output display
pd.set_option("display.precision", 2)

### 1 Read input files - postings and control codes

In [40]:
df_postings = pd.read_excel ('ACCSYSTEM_POSTINGS.xlsx', index_col='ID', header=0, sheet_name='POSTINGS')
df_controls = pd.read_excel ('ACCSYSTEM_POSTINGS.xlsx', index_col='ID', header=0, sheet_name='CONTROLS')
df_accounts = pd.DataFrame(index=['ID'], columns=['DOCUMENT DATE', 'DOCUMENT NUMBER', 'ACCOUNT', 'POSTING ENTRY', 'TAX RATE', 'OFFSET ACCOUNT', 'DEBIT', 'CREDIT'])
df_journal = pd.DataFrame(index=['ID'], columns=['DOCUMENT DATE', 'DOCUMENT NUMBER', 'ACCOUNT', 'POSTING ENTRY', 'TAX RATE', 'OFFSET ACCOUNT', 'DEBIT', 'CREDIT'])

In [41]:
df_postings.head()

,DOCUMENT DATE,DOCUMENT NUMBER,ACCOUNT DEBIT,AMOUNT DEBIT,ACCOUNT CREDIT,AMOUNT CREDIT,POSTING ENTRY,CONTROL CODE
ID,,,,,,,,
0,2021-01-01,DOC-1,1200,125000.00,9000,125000.00,EB,0
1,2021-01-01,DOC-2,1600,1000.00,9000,1000.00,EB,0
2,2021-01-01,DOC-3,808,15000.00,9000,15000.00,EB,0
3,2021-02-15,DOC-7,4806,758.89,1200,903.08,Aufwand X,1
4,2021-02-16,DOC-8,4806,179.52,1200,213.63,Aufwand X,1


In [42]:
df_postings.tail()

,DOCUMENT DATE,DOCUMENT NUMBER,ACCOUNT DEBIT,AMOUNT DEBIT,ACCOUNT CREDIT,AMOUNT CREDIT,POSTING ENTRY,CONTROL CODE
ID,,,,,,,,
51,2021-04-04,DOC-55,1200,10700.0,8400,10000.0,Erlös Z,4
52,2021-04-05,DOC-56,1200,10700.0,8400,10000.0,Erlös Z,4
53,2021-04-06,DOC-57,1200,10700.0,8400,10000.0,Erlös Z,4
54,2021-04-07,DOC-58,1200,10700.0,8400,10000.0,Erlös Z,4
55,2021-04-08,DOC-59,1200,10700.0,8400,10000.0,Erlös Z,4


In [43]:
df_controls

,CONTROL_CODE,COUNTRY,TAX_VARIANT,TAX_RATE,TAX_ACCOUNT,POSTING,GERMAN
ID,,,,,,,
0,0,DE,NO TAX,1.00,0,None,Keine Vor- oder Umsatzsteuer
1,1,DE,INPUT TAX,0.19,1576,Debit,Vorsteuer. Steuerkonto: Zugang im Soll
2,2,DE,INPUT TAX,0.07,1571,Debit,Vorsteuer. Steuerkonto: Zugang im Soll
3,3,DE,SALES TAX,0.19,1776,Credit,Umsatzsteuer. Steuerkonto: Zugang im Haben
4,4,DE,SALES TAX,0.07,1771,Credit,Umsatzsteuer. Steuerkonto: Zugang im Haben


### 2 Read and write debit account and debit amount

In [45]:
# Objectives: Read the posting file line by line and post the amount on the right side of each account
# The right side must be implemented into the posting list (debit account -> credit account)
# The control code determines tax account and side of posting (debit or credit)
# The account get an column which show the sending account (counter account?)
# Tax amount is calclulated with tax rate and net amount of sending account

# There is no dataframe for every single account, all accounts are in one dataframe
# For producing an account sheet this dataframe has to be filtered


# Loop through the dataframe: dataframe.index gives a list of the index elements 0..n
for posting_index in df_postings.index:

    # Read Control Code from Booking Input DataFrame
    current_control_code_postings = df_postings['CONTROL CODE'][posting_index]    
    
    # Process A: Post debit and credit account
    # Check case by case and set tax rate and tax account
    if current_control_code_postings == 0:
        tax_rate = df_controls.query('CONTROL_CODE==0')['TAX_RATE'].iloc[0]
        tax_account = df_controls.query('CONTROL_CODE==0')['TAX_ACCOUNT'].iloc[0]
    
    elif current_control_code_postings == 1:
        tax_rate = df_controls.query('CONTROL_CODE==1')['TAX_RATE'].iloc[0]
        tax_account = df_controls.query('CONTROL_CODE==1')['TAX_ACCOUNT'].iloc[0]
        
    elif current_control_code_postings == 2:
        tax_rate = df_controls.query('CONTROL_CODE==2')['TAX_RATE'].iloc[0]
        tax_account = df_controls.query('CONTROL_CODE==2')['TAX_ACCOUNT'].iloc[0]
        
    elif current_control_code_postings == 3:
        tax_rate = df_controls.query('CONTROL_CODE==3')['TAX_RATE'].iloc[0]
        tax_account = df_controls.query('CONTROL_CODE==3')['TAX_ACCOUNT'].iloc[0]
        
    elif current_control_code_postings == 4:
        tax_rate = df_controls.query('CONTROL_CODE==4')['TAX_RATE'].iloc[0]
        tax_account = df_controls.query('CONTROL_CODE==4')['TAX_ACCOUNT'].iloc[0]
        
    
    # 1 Read and write DEBIT account and amount to the datafrage df_accounts
    df_accounts = df_accounts.append({'DOCUMENT DATE': df_postings['DOCUMENT DATE'][posting_index],
                                      'DOCUMENT NUMBER': df_postings['DOCUMENT NUMBER'][posting_index],
                                      'ACCOUNT': df_postings['ACCOUNT DEBIT'][posting_index],
                                      'POSTING ENTRY': df_postings['POSTING ENTRY'][posting_index],
                                      'TAX RATE': tax_rate,
                                      'OFFSET ACCOUNT': df_postings['ACCOUNT CREDIT'][posting_index],
                                      'DEBIT': df_postings['AMOUNT DEBIT'][posting_index]
                                     },ignore_index = True)
    
 
     # 2 Read and write CREDIT account and amount to the dataframe df_accounts
    df_accounts = df_accounts.append({'DOCUMENT DATE': df_postings['DOCUMENT DATE'][posting_index],
                                      'DOCUMENT NUMBER': df_postings['DOCUMENT NUMBER'][posting_index],
                                      'ACCOUNT': df_postings['ACCOUNT CREDIT'][posting_index],
                                      'POSTING ENTRY': df_postings['POSTING ENTRY'][posting_index],
                                      'TAX RATE': tax_rate,
                                      'OFFSET ACCOUNT': df_postings['ACCOUNT DEBIT'][posting_index],
                                      'CREDIT': df_postings['AMOUNT CREDIT'][posting_index]
                                     },ignore_index = True)
    
    # Process B: Post tax account
    # Read Control Code from Booking Input DataFrame
    current_control_code_postings = df_postings['CONTROL CODE'][posting_index]
   
    # Case 1: Control Code = 0: Code for opening entries. No tax post, no tax account, no tax amount

    # Case 2: Debit Posting, Calculation from DEBIT AMOUNT
    if current_control_code_postings == 1:
        tax_rate = df_controls.query('CONTROL_CODE==1')['TAX_RATE'].iloc[0]
        tax_account = df_controls.query('CONTROL_CODE==1')['TAX_ACCOUNT'].iloc[0]
        
        df_accounts = df_accounts.append({'DOCUMENT DATE': df_postings['DOCUMENT DATE'][posting_index],
                                      'DOCUMENT NUMBER': df_postings['DOCUMENT NUMBER'][posting_index],
                                      'ACCOUNT': tax_account,
                                      'POSTING ENTRY': df_postings['POSTING ENTRY'][posting_index],
                                      'TAX RATE': tax_rate,
                                      'OFFSET ACCOUNT': df_postings['ACCOUNT CREDIT'][posting_index],
                                      'DEBIT': df_postings['AMOUNT DEBIT'][posting_index] * tax_rate
                                     },ignore_index = True)
       
    # Case 3: Debit Posting, Calculation from DEBIT AMOUNT    
    elif current_control_code_postings == 2:
        tax_rate = df_controls.query('CONTROL_CODE==2')['TAX_RATE'].iloc[0]
        tax_account = df_controls.query('CONTROL_CODE==2')['TAX_ACCOUNT'].iloc[0]
        
        df_accounts = df_accounts.append({'DOCUMENT DATE': df_postings['DOCUMENT DATE'][posting_index],
                                      'DOCUMENT NUMBER': df_postings['DOCUMENT NUMBER'][posting_index],
                                      'ACCOUNT': tax_account,
                                      'POSTING ENTRY': df_postings['POSTING ENTRY'][posting_index],
                                      'TAX RATE': tax_rate,
                                      'OFFSET ACCOUNT': df_postings['ACCOUNT CREDIT'][posting_index],
                                      'DEBIT': df_postings['AMOUNT DEBIT'][posting_index] * tax_rate
                                     },ignore_index = True)
        
    # Case 4: Credit Posting, Calculation from Credit AMOUNT    
    elif current_control_code_postings == 3:
        tax_rate = df_controls.query('CONTROL_CODE==3')['TAX_RATE'].iloc[0]
        tax_account = df_controls.query('CONTROL_CODE==3')['TAX_ACCOUNT'].iloc[0]
        
        df_accounts = df_accounts.append({'DOCUMENT DATE': df_postings['DOCUMENT DATE'][posting_index],
                                      'DOCUMENT NUMBER': df_postings['DOCUMENT NUMBER'][posting_index],
                                      'ACCOUNT': tax_account,
                                      'POSTING ENTRY': df_postings['POSTING ENTRY'][posting_index],
                                      'TAX RATE': tax_rate,
                                      'OFFSET ACCOUNT': df_postings['ACCOUNT DEBIT'][posting_index],
                                      'CREDIT': df_postings['AMOUNT CREDIT'][posting_index] * tax_rate
                                     },ignore_index = True)
        
    # Case 5: Credit Posting, Calculation from Credit AMOUNT     
    elif current_control_code_postings == 4:
        tax_rate = df_controls.query('CONTROL_CODE==4')['TAX_RATE'].iloc[0]
        tax_account = df_controls.query('CONTROL_CODE==4')['TAX_ACCOUNT'].iloc[0]
        
        df_accounts = df_accounts.append({'DOCUMENT DATE': df_postings['DOCUMENT DATE'][posting_index],
                                      'DOCUMENT NUMBER': df_postings['DOCUMENT NUMBER'][posting_index],
                                      'ACCOUNT': tax_account,
                                      'POSTING ENTRY': df_postings['POSTING ENTRY'][posting_index],
                                      'TAX RATE': tax_rate,
                                      'OFFSET ACCOUNT': df_postings['ACCOUNT DEBIT'][posting_index],
                                      'CREDIT': df_postings['AMOUNT CREDIT'][posting_index] * tax_rate
                                     },ignore_index = True)

In [46]:
# Delete NaN in Credit and Debit columns
df_accounts['DEBIT'].fillna(0, inplace=True)
df_accounts['CREDIT'].fillna(0, inplace=True)

# Set namr for index
df_accounts.index.name = 'ID'

In [47]:
# Show list of account postings
df_accounts

,DOCUMENT DATE,DOCUMENT NUMBER,ACCOUNT,POSTING ENTRY,TAX RATE,OFFSET ACCOUNT,DEBIT,CREDIT
ID,,,,,,,,
0,NaT,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,2021-01-01,DOC-1,1200,EB,1.00,9000,125000.0,0.0
2,2021-01-01,DOC-1,9000,EB,1.00,1200,0.0,125000.0
3,2021-01-01,DOC-2,1600,EB,1.00,9000,1000.0,0.0
4,2021-01-01,DOC-2,9000,EB,1.00,1600,0.0,1000.0
...,...,...,...,...,...,...,...,...
161,2021-04-07,DOC-58,8400,Erlös Z,0.07,1200,0.0,10000.0
162,2021-04-07,DOC-58,1771,Erlös Z,0.07,1200,0.0,700.0
163,2021-04-08,DOC-59,1200,Erlös Z,0.07,8400,10700.0,0.0


### 3 Write output to an Excel file

In [48]:
with pd.ExcelWriter('OUTPUT.xlsx') as writer:
    df_accounts.to_excel(writer, sheet_name='ACCOUNTS')

### 4 Test some groupings

In [49]:
df_accounts.query('ACCOUNT==1771')

,DOCUMENT DATE,DOCUMENT NUMBER,ACCOUNT,POSTING ENTRY,TAX RATE,OFFSET ACCOUNT,DEBIT,CREDIT
ID,,,,,,,,
129,2021-03-27,DOC-47,1771,Erlös Z,0.07,1200,0.0,700.0
132,2021-03-28,DOC-48,1771,Erlös Z,0.07,1200,0.0,700.0
135,2021-03-29,DOC-49,1771,Erlös Z,0.07,1200,0.0,700.0
138,2021-03-30,DOC-50,1771,Erlös Z,0.07,1200,0.0,700.0
141,2021-03-31,DOC-51,1771,Erlös Z,0.07,1200,0.0,700.0
144,2021-04-01,DOC-52,1771,Erlös Z,0.07,1200,0.0,700.0
147,2021-04-02,DOC-53,1771,Erlös Z,0.07,1200,0.0,700.0
150,2021-04-03,DOC-54,1771,Erlös Z,0.07,1200,0.0,700.0
153,2021-04-04,DOC-55,1771,Erlös Z,0.07,1200,0.0,700.0


In [50]:
# Check number of resulting postings
df_accounts_grouped = df_accounts.groupby('ACCOUNT')['ACCOUNT'].count()

In [51]:
df_accounts_grouped

ACCOUNT
808      1
1200    54
1576    28
1600     1
1771    13
1776    12
4806    28
8400    25
9000     3
Name: ACCOUNT, dtype: int64

In [55]:
# Show list of DataFrame index
df_postings.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55],
           dtype='int64', name='ID')